# Intro to Pytorch

in this notebook, we are going to work with Fashion-MNIST datset that is a set of 28x28 greyscale images of clothes. We are going to build a neurol ntwrork that help us identifin  clotes in in. in the dataset,  each image is 28x28 which is a total of 784 pixels, and there are 10 classes.

In [ ]:
#import the libraies
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import  datasets, transforms


In [7]:
#defining a transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

#dowloand and test datasets
testdataset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testdataset, batch_size=64, shuffle=True)

In [8]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # Dropout module with 0.2 drop probability
        self.dropout = nn.Dropout(p = 0.2)
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x
        

```python
# we CAN DEFINE A NETWORK SEQUENTIAL MODEULE MORE EASYILY
#you can use both of them
model_seq = nn.Sequential(nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64,10),
                      nn.LogSoftmax())
```

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#let's go this time with Classifier()
model = Classifier()

#define the criterion and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
model.to(device)

Classifier(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2)
)

In [10]:
# Train the network here
epochs =5

train_losses , test_losses = [],[]
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        #forword the model
        images = images.view(images.shape[0], -1)
        log_ps = model(images)
        #calculate the loss
        loss = criterion(log_ps, labels)
        
        #make the optim zero for each loop
        optimizer.zero_grad()
        #backward propagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        test_loss = 0
        accuracy =0 
        with torch.no_grad():
            model.eval()
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                log_ps = model(images)
                test_loss +=criterion(log_ps, labels)
                
                #calculate the accuracy
                #predic the label uisng tets data
                ps = torch.exp(log_ps)
                top_p , top_classes = ps.topk(1, dim=1)
                equels = top_classes == labels.view(*top_classes.shape)
                accuracy = torch.mean(equels.type(torch.FloatTensor))
        
        model.train()
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))
        
        print("Epochs: {}/{}..".format(e+1, epochs),
              "Trainig Loss: {:.3f}..".format(train_losses[-1]),
              "Test Loss: {:.3f}..".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader))
             )
        

Epochs: 1/5.. Trainig Loss: 0.610.. Test Loss: 0.469.. Test Accuracy: 0.006
Epochs: 2/5.. Trainig Loss: 0.480.. Test Loss: 0.450.. Test Accuracy: 0.005
Epochs: 3/5.. Trainig Loss: 0.450.. Test Loss: 0.407.. Test Accuracy: 0.006
Epochs: 4/5.. Trainig Loss: 0.431.. Test Loss: 0.408.. Test Accuracy: 0.005
Epochs: 5/5.. Trainig Loss: 0.421.. Test Loss: 0.397.. Test Accuracy: 0.006


In [ ]:
\